In [1]:
# !pip install transformers deepspeed-mii --upgrade

In [2]:
# no MII server is running
!sudo netstat -ltnp | grep -w ':50050' 

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
import mii
mii_configs = {"tensor_parallel": 1, "dtype": "fp16"}
mii.deploy(task="text-generation",
           model="bigscience/bloom-560m",
           deployment_name="bloom560m_deployment_async",
           mii_config=mii_configs)

[2022-11-17 05:36:45,148] [INFO] [deployment.py:87:deploy] ************* MII is using DeepSpeed Optimizations to accelerate your model *************
[2022-11-17 05:36:45,334] [INFO] [server_client.py:219:_initialize_service] MII using multi-gpu deepspeed launcher:
 ------------------------------------------------------------
 task-name .................... text-generation 
 model ........................ bigscience/bloom-560m 
 model-path ................... /tmp/mii_models 
 port ......................... 50050 
 provider ..................... hugging-face 
 ------------------------------------------------------------
[2022-11-17 05:36:46,519] [WARNING] [runner.py:179:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2022-11-17 05:36:46,548] [INFO] [runner.py:508:main] cmd = /home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 

In [6]:
generator = mii.mii_query_handle("bloom560m_deployment_async")
result = generator.query({"query": ["DeepSpeed is", "Seattle is"]}, do_sample=True, max_new_tokens=30)
result

response: "DeepSpeed is not an option for us. Heavier loads are available with the Viper and the Dodge Ram DCTs.\nWe were lucky enough to get"
response: "Seattle is the world’s largest, and most populous, metropolitan area. In the fall it holds off on Christmas festivities. After the summer holiday, it"
time_taken: 0.924416542
model_time_taken: -1

In [7]:
mii.terminate("bloom560m_deployment_async")

[2022-11-17 05:37:40,128] [INFO] [terminate.py:8:terminate] Terminating server for bloom560m_deployment_async
[2022-11-17 05:37:40,783] [INFO] [launch.py:318:sigkill_handler] Killing subprocess 7252
[2022-11-17 05:37:40,783] [ERROR] [launch.py:324:sigkill_handler] ['/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python', '-m', 'mii.launch.multi_gpu_server', '--task-name', 'text-generation', '--model', 'bigscience/bloom-560m', '--model-path', '/tmp/mii_models', '--port', '50050', '--ds-optimize', '--provider', 'hugging-face', '--config', 'eyJ0ZW5zb3JfcGFyYWxsZWwiOiAxLCAicG9ydF9udW1iZXIiOiA1MDA1MCwgImR0eXBlIjogImZwMTYiLCAiZW5hYmxlX2N1ZGFfZ3JhcGgiOiBmYWxzZSwgImNoZWNrcG9pbnRfZGljdCI6IG51bGwsICJkZXBsb3lfcmFuayI6IFswXSwgInRvcmNoX2Rpc3RfcG9ydCI6IDI5NTAwLCAiaGZfYXV0aF90b2tlbiI6IG51bGwsICJyZXBsYWNlX3dpdGhfa2VybmVsX2luamVjdCI6IHRydWUsICJwcm9maWxlX21vZGVsX3RpbWUiOiBmYWxzZSwgInNraXBfbW9kZWxfY2hlY2siOiBmYWxzZX0='] exits with return code = -15
